i don't plan to make any more changes unless for bug fix.

here are three models, saved in `trial/model`

- `model_o` is the original transformer
- `model_e` is the new model with softmax attention, `e` for exponential
- `model_q` is the new model with squared attention, `q` for square

i like `model_e` the best, but judge for yourself :D see `pred/diff`

In [1]:
from model_e import Transformer
trial = 'e'

In [2]:
from util import PointedIndex
from util_io import encode, decode
import numpy as np
import tensorflow as tf

In [3]:
len_cap = 256
ckpt = 34990

In [4]:
model = Transformer.new().data(len_cap= len_cap)
forcing = model.forcing(trainable= False) # the model in teacher forcing mode
autoreg = model.autoreg(trainable= False) # the model in autoregressive mode

In [5]:
saver = tf.train.Saver()
sess = tf.InteractiveSession()
saver.restore(sess, "trial/model/{}{}".format(trial, ckpt))

INFO:tensorflow:Restoring parameters from trial/model/e34990


In [6]:
idx_src = PointedIndex(np.load("trial/data/index_src.npy").item())
idx_tgt = PointedIndex(np.load("trial/data/index_tgt.npy").item())

def auto(s, m= autoreg, idx_src= idx_src, idx_tgt= idx_tgt, len_cap= len_cap):
    # encode the sentence as a numpy array
    # it's automatically padded at the beginning and the end
    src = np.array(encode(idx_src, s))
    # reshape the array into a batch with one instance
    src.shape = 1, -1
    # fetch the prediction
    # tgt needs to be fed the first step (the padding at the beginning)
    # len_tgt is the maximum steps to unroll
    pred = m.pred.eval({m.src: src, m.tgt: src[:,:1], m.len_tgt: len_cap})
    # take the only instance from the batch
    pred = pred[0]
    # decode the prediction
    return decode(idx_tgt, pred)

# to run the forcing model, remember to feed a encoded target sentence
# and DON'T feed len_tgt
# cuz that's simply the length of the encoded target minus the beginning

In [7]:
auto("Repeat !")

'Repeat Repeat Repeat Repeat .'

if you wanna know more about the training process for your own experiments,
see `src/train_e.py` among others.
(they have no difference except for which model to load).
i commented out the code for training.
it has code for profiling the graph, but i also copied the result to `src/trial/graph`.

to understand the implementation of the model, see `model_e.py` among others.
(there are minor variations.  the `diff` command would be useful.)

don't hesitate to ask me if you have questions about my code.

the attention tensors in the teacher forcing model are these:
```
encode_forcing/layer1/att/attention/Reshape_1:0
encode_forcing/layer2/att/attention/Reshape_1:0
decode_forcing/layer1/att/causal/Reshape_1:0
decode_forcing/layer1/att/attention/Reshape_1:0
decode_forcing/layer2/att/causal/Reshape_1:0
decode_forcing/layer2/att/attention/Reshape_1:0
```

the attention tensors in the autoregressive model are these:
```
encode_autoreg/layer1/att/attention/Reshape_1:0
encode_autoreg/layer2/att/attention/Reshape_1:0
decode_autoreg/autoreg/layer1/att/causal/Reshape_1:0
decode_autoreg/autoreg/layer1/att/attention/Reshape_1:0
decode_autoreg/autoreg/layer2/att/causal/Reshape_1:0
decode_autoreg/autoreg/layer2/att/attention/Reshape_1:0
```

however tensorflow forbids the decoder attentions to be fetched, cuz they are inside of a loop.
if you want these weights, you can run the forcing model repeatedly, feeding its previous predictions back each time, and accumulate the results.